In [2]:
import pandas as pd
from csvvalidator import *
import datetime as DT
import numpy as np

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [6]:
#Validate data
field_names = (
               'Ticket number',
               'Issue Date',
               'Latitude',
               'Longitude',
               'Issue time',
               'Violation Description'
               )
validator = CSVValidator(field_names)
# basic header and record length checks
validator.add_header_check('EX1', 'bad header')
validator.add_record_length_check('EX2', 'unexpected record length')

In [3]:
df = pd.read_csv('../parking_citation_2020_2022.csv')

/tmp/ddlin/23123757/ipykernel_3360257/3233484126.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../parking_citation_2020_2022.csv')


In [4]:
#updating formatting so that I can translate issue date to datetime
df['Issue Date'] = df[df['Issue Date'].notnull()]['Issue Date'].apply(lambda x: x.split('T')[0])
df['Issue Date'] = pd.to_datetime(df['Issue Date'], infer_datetime_format=True)

In [5]:
#pad anything that is less than 4 digits then isolate just the hours
df['Issue time'] = df['Issue time'].astype(str)
df['Issue time'] = df['Issue time'].apply(lambda x: x.split('.')[0])
df['Issue time'] = df[df['Issue time'].notnull()]['Issue time'].apply(lambda x: x.zfill(4))
df['Issue Hour'] = df[df['Issue time']!='0nan']['Issue time'].apply(lambda x: DT.datetime.strptime(x,'%H%M').hour)

#clean lat lon
df['Latitude'] = np.where(df['Latitude']==99999.000, np.nan, df['Latitude'])
df['Longitude'] = np.where(df['Longitude']==99999.000, np.nan, df['Longitude'])

#string for ticketnum
df['Ticket number'] = df['Ticket number'].astype(str)

In [6]:
#Updating the Lat Lon
import pyproj
pm = '+proj=lcc +lat_1=34.03333333333333 +lat_2=35.46666666666667 +lat_0=33.5 +lon_0=-118 +x_0=2000000 +y_0=500000.0000000002 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'
x1m,y1m = df['Latitude'].values, df['Longitude'].values
x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)
df['Latitude']=x2m
df['Longitude']=y2m

/data/homezvol2/ddlin/mambaforge-pypy3/envs/plotly/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ddlin/23123757/ipykernel_3360257/1413484023.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)


In [ ]:
# df = pd.read_csv("../cleaned_2022_parking_citation.csv")

In [7]:
df.columns

Index(['Ticket number', 'Issue Date', 'Issue time', 'Meter Id', 'Marked Time',
       'RP State Plate', 'Plate Expiry Date', 'VIN', 'Make', 'Body Style',
       'Color', 'Location', 'Route', 'Agency', 'Violation code',
       'Violation Description', 'Fine amount', 'Latitude', 'Longitude',
       'Agency Description', 'Color Description', 'Body Style Description',
       'Issue Hour'],
      dtype='object')

In [8]:
df['Body Style Description'].value_counts(), df['Body Style'].value_counts()

(PASSENGER CAR    4806029
 PICK-UP TRUCK     165537
 VAN                91295
 COMMERCIAL         76050
 TRUCK              67966
 TRAILER            38114
 MOTOR HOME         20603
 BUS                 1680
 LIMOUSINE            239
 Name: Body Style Description, dtype: int64,
 PA    4806029
 PU     165537
 VN      91295
 TK      67966
 CM      64129
        ...   
 RB          1
 02          1
 FB          1
 RX          1
 AU          1
 Name: Body Style, Length: 133, dtype: int64)

In [9]:
df.shape

(5308775, 23)

In [10]:
data_df = df[[ 'RP State Plate', 'Make', 'Body Style Description', 'Color Description', 'Agency Description', 'Issue Hour', 'Latitude', 'Longitude', 'Violation Description']]
data_df.head()

,RP State Plate,Make,Body Style Description,Color Description,Agency Description,Issue Hour,Latitude,Longitude,Violation Description
0,CA,MAZD,PASSENGER CAR,GREY,54 - DOT - HOLLYWOOD,13.000,-118.340,34.100,WHITE ZONE
1,CA,LEXS,PASSENGER CAR,BLACK,51 - DOT - WESTERN,16.000,-118.471,33.984,NaN
2,CA,MERZ,PASSENGER CAR,BLACK,51 - DOT - WESTERN,16.000,-118.354,34.062,NaN
3,CA,PORS,PASSENGER CAR,BLACK,55 - DOT - SOUTHERN,2.000,-118.335,34.033,STANDNG IN ALLEY
4,CA,RROV,PASSENGER CAR,GREY,51 - DOT - WESTERN,1.000,-118.396,34.027,PARKED ON SIDEWALK


In [11]:
data_df = data_df[~data_df.isna().any(axis=1)]
data_df.shape

(4566737, 9)

In [12]:
from sklearn.cluster import KMeans

# Assuming your dataframe is named 'data' and has columns 'Latitude' and 'Longitude'
coordinates = data_df[['Latitude', 'Longitude']].values

# Create a KMeans clustering model with 100 clusters
kmeans = KMeans(n_clusters=500, random_state=0).fit(coordinates)

# Add cluster labels to the original dataframe
data_df['Cluster'] = kmeans.labels_

/data/homezvol2/ddlin/mambaforge-pypy3/envs/plotly/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [13]:
data_df.head()

,RP State Plate,Make,Body Style Description,Color Description,Agency Description,Issue Hour,Latitude,Longitude,Violation Description,Cluster
0,CA,MAZD,PASSENGER CAR,GREY,54 - DOT - HOLLYWOOD,13.000,-118.340,34.100,WHITE ZONE,155
3,CA,PORS,PASSENGER CAR,BLACK,55 - DOT - SOUTHERN,2.000,-118.335,34.033,STANDNG IN ALLEY,256
4,CA,RROV,PASSENGER CAR,GREY,51 - DOT - WESTERN,1.000,-118.396,34.027,PARKED ON SIDEWALK,283
5,CA,VOLK,PASSENGER CAR,GREY,56 - DOT - CENTRAL,1.000,-118.213,34.043,RED ZONE,348
6,CA,HOND,PASSENGER CAR,BLACK,53 - DOT - VALLEY,2.000,-118.446,34.229,PREFERENTIAL PARKING,440


In [14]:
data_df.to_csv('../cleaned_2020_2022_parking_citation.csv', index=False)
# data_df = pd.read_csv('../cleaned_2020_2022_parking_citation.csv')
# data_df.head()

In [26]:
# Get the value counts
top10_violations = data_df['Violation Description'].value_counts().index[:10]
top10_df = data_df[data_df['Violation Description'].isin(top10_violations)]
top10_df.to_csv('../top10_violations_2020_2022.csv', index=False)

# Remove rows with 'Violation Description' values in 'to_remove'
# data_df = data_df[~data_df['Violation Description'].isin(to_remove)]


In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import precision_score, recall_score
# from sklearn.utils import class_weight
import time

start_time = time.time()

# Check cuda
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)


# 1. Data Preparation
# ---------------------
data = pd.read_csv("../cleaned_2020_2022_parking_citation.csv")
features = [ 'RP State Plate', 'Make', 'Body Style Description', 'Color Description', 'Agency Description', 'Issue Hour', 'Cluster']
X = data[features]

# One-hot encode all features
ohe = OneHotEncoder(sparse=False, dtype=int)
X_encoded = ohe.fit_transform(X)
encoded_columns = ohe.get_feature_names_out(features)  # Get column names for the encoded columns
X = pd.DataFrame(X_encoded, columns=encoded_columns)

# Extract target variable and encode it
y = LabelEncoder().fit_transform(data['Violation Description'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Now, after the split, calculate the class weights
# class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
batch_size = 256
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 2. Model Definition
# ---------------------
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)  # New layer
        self.layer4 = nn.Linear(32, output_dim)  # Modified from layer3 to layer4

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))  # Activation for the new layer
        x = self.layer4(x)  # Final layer
        return x

model = SimpleNN(X_train.shape[1], len(set(y_train))).to(device)
# criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initializing lists to store precision and recall values for each epoch
epoch_precisions = []
epoch_recalls = []
epochs = 10
# 3. Training
# ------------
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    # 4. Evaluation
    # ---------------
    with torch.no_grad():
        y_pred = model(X_train_tensor)
        _, predicted = torch.max(y_pred, 1)
        
        # Convert tensors to numpy arrays for sklearn metrics
        true_labels = y_train_tensor.cpu().numpy()
        pred_labels = predicted.cpu().numpy()
        
        # Calculate precision and recall
        precision = precision_score(true_labels, pred_labels, average='macro')
        recall = recall_score(true_labels, pred_labels, average='macro')
        epoch_precisions.append(precision)
        epoch_recalls.append(recall)

        accuracy = (predicted == y_train_tensor).sum().item() / len(y_train_tensor)
        print(f"Training Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")

True
cuda:0


/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10, Loss: 2.056368350982666


/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training Accuracy: 0.4677, Precision: 0.1139, Recall: 0.0820
Epoch 2/10, Loss: 1.9851702451705933


...

Training Accuracy: 0.4823, Precision: 0.1532, Recall: 0.0995
Epoch 9/10, Loss: 2.7009828090667725


/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training Accuracy: 0.4835, Precision: 0.1734, Recall: 0.1033
Epoch 10/10, Loss: 2.162100315093994


/home/tealeave/mambaforge/envs/d2l/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training Accuracy: 0.4829, Precision: 0.1758, Recall: 0.1015
Total execution time: 178.37 seconds


In [50]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# import pandas as pd

# # Load your data
# data = pd.read_csv("cleaned_2022_parking_citation.csv")

# # Preprocess the data (similar to what we did above)
# data = data.dropna(subset=['Violation Description'])
# features = ['RP State Plate', 'Make', 'Body Style', 'Color Description', 'Agency Description', 'Issue Hour']
# X = data[features]
# X = X.fillna('Unknown')

# label_encoders = {}
# for col in X.select_dtypes(include=['object']).columns:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     label_encoders[col] = le

# scaler = StandardScaler()
# X[['Issue time', 'Issue Hour']] = scaler.fit_transform(X[['Issue time', 'Issue Hour']])
# y = LabelEncoder().fit_transform(data['Violation Description'])

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the XGBoost classifier
# clf = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y_train)), random_state=42, use_label_encoder=False)
# clf.fit(X_train, y_train, eval_metric='mlogloss')

# # Get training accuracy (you can also get testing accuracy if you wish)
# train_accuracy = clf.score(X_train, y_train)
# print(f"Training Accuracy: {train_accuracy}")